In [220]:
import random
from pprint import pprint
import copy

def generate_backend_code(IR, num_PEs, cycle_times):
    # Step 2: Receive IR, number of PEs, and cycle times

    # Step 3: Assign initial tasks to PEs
    assignments = initial_assignment(IR, num_PEs)
    
    #Step 4:
    execution_times = calculate_execution_times(assignments, cycle_times)

    # Step 5: Check workload imbalance
    max_execution_time = max(execution_times)
    min_execution_time = min(execution_times)
    cur_imbalance = max_execution_time - min_execution_time


    threshold = 0.1  # Workload imbalance threshold
    iteration = 0
    max_iterations = 1000  # Maximum number of iterations


    #Repeat 4-6
    while True:
        
        
        # Step 6: Task migration or swapping strategy
        new_assignments = rebalance_workload(assignments, execution_times)

        # Step 4: Calculate execution cost for each PE
        execution_times = calculate_execution_times(new_assignments, cycle_times)

        # Step 5: Check workload imbalance
        max_execution_time = max(execution_times)
        min_execution_time = min(execution_times)
        new_imbalance = max_execution_time - min_execution_time

        print(f"Iteration: {iteration}\t New Imbalance: {new_imbalance}, Current Imbalance: {cur_imbalance}")
        """ for idx, (old, new )in  enumerate(zip(assignments, new_assignments)):
            print(f"instrc: {idx}, {old} => {new}") """
        if new_imbalance > cur_imbalance:
            print(f"Stopping with an Imbalance of {cur_imbalance}")
            
            break  # Terminate if workload is balanced within threshold or maximum iterations reached
        cur_imbalance = new_imbalance
        assignments = new_assignments
        
        iteration += 1 
    synced_tasks = sync(assignments, cycle_times, IR)
    for pe_id, assigned_tasks in enumerate(synced_tasks):
        # Step 8: Generate output code for each PE
        code = generate_code(assigned_tasks, cycle_times)

        # Step 9: Dump output code to files
        dump_code_to_file(code, pe_id)

def initial_assignment(tasks, num_PEs):
    # Assign initial tasks to PEs
    
    assignments = [[] for _ in range(num_PEs)]
    task_index = 0

    for task in tasks:
        pe_id = task_index % num_PEs
        assignments[pe_id].append(task)
        task_index += 1

    
    return assignments

def calculate_execution_times(assignments, cycle_times):
    # Calculate execution time for each PE
    execution_times = []

    for assigned_tasks in assignments:
        execution_time = 0
        for task in assigned_tasks:
            if task:
                task_name = task[0]
                if task_name in cycle_times:
                    execution_time += cycle_times[task_name]
        execution_times.append(execution_time)

    return execution_times

def rebalance_workload(assignments, execution_times):

    new_assignments = copy.deepcopy(assignments)
    
    # Perform task migration or swapping to balance workload
    max_index = execution_times.index(max(execution_times))
    min_index = execution_times.index(min(execution_times))

    # Swap a task from the most loaded PE to the least loaded PE
    task_to_swap = new_assignments[max_index].pop(0)
    new_assignments[min_index].append(task_to_swap)

    return new_assignments

def sync(assignments,cycle_times,IR):\


    print("\n\n")
    sync_code = [[] for _ in range(len(assignments))]
    hash = {}
    instruction_cycle_times = [cycle_times[task[0]] for task in IR]
    for pos, instruc in enumerate(IR):
        hash[instruc] = pos
        

    live_time = [0 for _ in range(len(assignments)) ]
    current_instruction = ["NOP"for _ in range(len(assignments)) ]
    numerical_assignment = [[(hash[task],task[-1]) for task in tasks] for tasks in assignments ]

    print(instruction_cycle_times)
    print(assignments)
    print(numerical_assignment)

    instructions_done = set()
    instructions_done.add(None)
    print("here:",(i for i in range(len(IR))) not in instructions_done)
    cycle = 1
    while len(IR) != len(instructions_done)-1:

        for idx, (current, count) in enumerate(zip(current_instruction,live_time)):
            #print(idx, current, count)
            if current != "NOP":
                if count <= 1:
                    instructions_done.add(current)
                    current_instruction[idx] = "NOP"
                else:
                    live_time[idx] -= 1

        for assignment_id, tasks in enumerate(numerical_assignment):
            
            if current_instruction[assignment_id] == "NOP":
                for task in tasks:
                    #print(assignment_id, task, type(task[1]), task[0] not in instructions_done and task[1] in instructions_done)
                    if task[0] not in instructions_done and all(num in instructions_done for num in task[1]):
                        print("Added ",task[0])
                        current_instruction[assignment_id] = task[0]  
                        live_time[assignment_id] = instruction_cycle_times[task[0]]
                        sync_code[assignment_id].append(IR[task[0]])

            if current_instruction[assignment_id] == "NOP" and len(IR) != len(instructions_done)-1:
               sync_code[assignment_id].append("(NOP)")


        

        print(f'Cycle {cycle}')
        print(instructions_done)
        print(current_instruction)
        print(live_time)
        print()
        
        cycle += 1

        

        #sleep(1)
    #pprint(sync_code)
    return  sync_code 
        
def generate_code(tasks, cycle_times):
    # Generate output code for a given set of tasks
    code = ""
    cycle_times['('] = 1
    #print("final tasks:",tasks)
    for task in tasks:
        if task:
            #print(task)
            for idx in range(cycle_times[task[0]]):
               
                    code += (str(task) + "\n")  if idx == 0 else "\n"

    return code

def dump_code_to_file(code, pe_id):
    # Dump generated code to a file for a specific PE
    filename = f"PE_{pe_id}_code.txt"

    with open(filename, "w") as file:
        file.write(code)

# Example usage
IR = [('LOAD', 't1', 'x', (None,)),
 ('ADD', 't2', 't1', '4', (0,)),
 ('MUL', 't3', 't1', '8', (0,)),
 ('DIV', 't4', 't3', 't2', (2, 1)),
 ('STORE', 'y', 't4', (3,))]

print(IR)
num_PEs = 3 # Number of processing elements

cycle_times = {
    'LOAD': 1,
    'ADD': 1,
    'MUL': 4,
    'DIV': 8,
    'STORE': 1
}

generate_backend_code(IR, num_PEs, cycle_times)





[('LOAD', 't1', 'x', (None,)), ('ADD', 't2', 't1', '4', (0,)), ('MUL', 't3', 't1', '8', (0,)), ('DIV', 't4', 't3', 't2', (2, 1)), ('STORE', 'y', 't4', (3,))]
Iteration: 0	 New Imbalance: 5, Current Imbalance: 7
Iteration: 1	 New Imbalance: 11, Current Imbalance: 5
Stopping with an Imbalance of 5



[1, 1, 4, 8, 1]
[[('DIV', 't4', 't3', 't2', (2, 1))], [('ADD', 't2', 't1', '4', (0,)), ('STORE', 'y', 't4', (3,)), ('LOAD', 't1', 'x', (None,))], [('MUL', 't3', 't1', '8', (0,))]]
[[(3, (2, 1))], [(1, (0,)), (4, (3,)), (0, (None,))], [(2, (0,))]]
here: True
Added  0
Cycle 1
{None}
['NOP', 0, 'NOP']
[0, 1, 0]

Added  1
Added  2
Cycle 2
{0, None}
['NOP', 1, 2]
[0, 1, 4]

Cycle 3
{0, 1, None}
['NOP', 'NOP', 2]
[0, 1, 3]

Cycle 4
{0, 1, None}
['NOP', 'NOP', 2]
[0, 1, 2]

Cycle 5
{0, 1, None}
['NOP', 'NOP', 2]
[0, 1, 1]

Added  3
Cycle 6
{0, 1, 2, None}
[3, 'NOP', 'NOP']
[8, 1, 1]

Cycle 7
{0, 1, 2, None}
[3, 'NOP', 'NOP']
[7, 1, 1]

Cycle 8
{0, 1, 2, None}
[3, 'NOP', 'NOP']
[6, 1, 1]

Cycle 9
{0

In [165]:
instructions_done = set()
instructions_done.add(2)
instructions_done.add(1)
x = (2,1)
all(num in instructions_done for num in x)

True